In [ ]:
'''import cv2
import numpy as np
from skimage import io, color
import matplotlib.pyplot as plt
%matplotlib inline


def get_sift_feature(im):
    """
    Extract the SIFT features from an input image.

    Parameters:
        im (numpy.ndarray): An input image in float32 format with shape (H, W, 3).

    Returns:
        A 1D array of the mean of the SIFT descriptors. If no descriptors are found,
        returns a zero array of length 128.
    """
    gray_im = color.rgb2gray(im)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)
    if descriptors is None:
        return np.zeros((128,))
    return np.mean(descriptors, axis=0)



img = cv2.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bae8f.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)


img1 = io.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bb10e.jpg")
io.imshow(img1)
io.show()
print(get_sift_feature(img1))'''

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage import io, color
#from sklearn.feature_extraction import image
#from sklearn.feature_extraction.image import extract_surf_features
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
%matplotlib inline

# Get the directory path
directory_path = "/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/data/train_ims" #change path

# List filenames in alphabetical order
filenames = sorted(os.listdir(directory_path))

In [ ]:
def get_sift_feature(im):
    """
    Extract the SIFT features from an input image.

    Parameters:
        im (numpy.ndarray): An input image in float32 format with shape (H, W, 3).

    Returns:
        A 1D array of the mean of the SIFT descriptors. If no descriptors are found,
        returns a zero array of length 128.
    """
    gray_im = color.rgb2gray(im)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)
    if descriptors is None:
        return np.zeros((128,))
    return np.mean(descriptors, axis=0) 

def get_orb_feature(im):
    """
    Extract the ORB features from an input image.

    Parameters:
        im (numpy.ndarray): An input image in float32 format with shape (H, W, 3).

    Returns:
        A 1D array of the mean of the ORB descriptors. If no descriptors are found,
        returns a zero array of length 32.
    """
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()
    _, descriptors = orb.detectAndCompute(gray_im, None)

    if descriptors is None:
        return np.zeros((32,))

    # Calculate the mean of the ORB descriptors
    mean_descriptor = np.mean(descriptors, axis=0)

    return mean_descriptor

In [ ]:
initial_images = []
train_df = pd.read_csv("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/data/train.csv") #change path
labels = train_df["label"].to_numpy()[:1000] #sliced to 1000
print(labels)


for i in range(1000): #only checking first 1000 images
    image = io.imread(f"{directory_path}/{filenames[i]}")
    initial_images.append(image)

print(initial_images)

In [ ]:
orb_features = []
for image in initial_images:
    orb_features.append(get_orb_feature(image)) #checking for ORB feature instead

orb_features = np.array(orb_features)
labels = np.array(labels)

param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svc = SVC()
clf = GridSearchCV(svc, param_grid, cv=5)

# Train the SVM classifier
clf.fit(orb_features, labels)

# Print the best hyperparameters and score
print("Best parameters:", clf.best_params_)
print("Best score:", clf.best_score_)